# Notebook 1: MED-PC Extracting the Recording Data and Metadata

# Importing the Python Libraries

In [1]:
import sys
import glob
import collections
from collections import defaultdict
import os
from datetime import datetime
import pathlib

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from medpc2excel.medpc_read import medpc_read
from moviepy.editor import *

In [3]:
# setting path
sys.path.append('../../src')

In [4]:
# All the libraries that were created for this repository
import extract.dataframe
import processing.tone
import extract.metadata

# Getting the Metadata from all the MED-PC Recording Files

- Every MED-PC Recording file has Metadata about time, data, subject, group, experiment name, script used, and the MED-PC box number on the top of the recording file. We will first extract that information
    - Metadata can be background information or context about data(or files) that is often different from the actual data that is intended to be recorded.  
        For more information on Metadata: https://en.wikipedia.org/wiki/Metadata

- The path of the directory that this notebook is in. This will be where relative paths will be based off of

In [5]:
current_working_directory = os.getcwd()

In [6]:
current_working_directory

'/nancy/projects/dominance_strain_comparison/results/2023_05_02_rc_sleap_analysis'

- All the other files in this directory. If you want to use a folder in here such as the `data` folder, you'd type `./data/{name_of_folder}` where you replace `{name_of_folder}` with the name of the folder without the `{}`. You will do this in the cells following the one below
    - The `./` means the path will reference the current directory that the command is being used from

In [7]:
os.listdir(current_working_directory)

['proc',
 '.ipynb_checkpoints',
 '03_making_inference.ipynb',
 '2022_10_27_pilot_1_2_3_reward_competition_video_analysis',
 '01_extracting_recording_data_and_metadata.ipynb',
 'data',
 'rc_distance_analysis.ipynb',
 '02_get_tone_times.ipynb',
 '03_coordinate_analysis.ipynb']

## NOTE: If you are using your own data, the following path must be changed to the directory where your MED-PC recording files are located(if they are not in the specied folder). It is recommended to create a sub folder in the `data` folder(that is in the same folder as this notebook) for each group of session you want to process and put the recording files in there

- Use the cell below instead if you're using your own data. You must change the cell below by clicking on the cell and then pressing the `esc` key. Then press the `Y` button on your keyboard. If you want to switch it back do the same but press the `R` button instead. This switches it from a cell that is run as if it's code, or back to a cell where it's just treated as text
    - **NOTE: If you are using a different folder, then change the path in the cell below**
    - The asterisk is called a wild card which will tell the computer to find all files/folders that match this pattern. The `*` means that any pattern of any length can be replaced with it. The `**` tells the computer to look in all folders that are in the specified folder. For more information: https://linuxhint.com/bash_wildcard_tutorial/
    - For more information on finding the path of your folder that contains the recording files: https://www.computerhope.com/issues/ch001708.htm

In [8]:
all_med_pc_file = glob.glob("./data/med_pc_recording_files/*.txt")

In [9]:
all_med_pc_file[:10]

['./data/med_pc_recording_files/2022-10-03_11h07m_Subject 4.3v4.4.txt',
 './data/med_pc_recording_files/2022-10-04_09h56m_Subject 4.1v4.4.txt',
 './data/med_pc_recording_files/2022-10-03_09h45m_Subject 2.1v2.2.txt',
 './data/med_pc_recording_files/2022-10-03_09h45m_Subject 1.3v1.4.txt',
 './data/med_pc_recording_files/2022-10-03_11h45m_Subject 5.3v5.4.txt',
 './data/med_pc_recording_files/2022-10-03_16h53m_Subject 5.1v5.4.txt',
 './data/med_pc_recording_files/2022-10-04_15h37m_Subject 1.3v1.4.txt',
 './data/med_pc_recording_files/2022-10-04_14h51m_Subject 3.1v3.2.txt',
 './data/med_pc_recording_files/2022-10-04_14h51m_Subject 4.1v4.2.txt',
 './data/med_pc_recording_files/2022-10-04_13h08m_Subject 2.2v2.4.txt']

- Example of what the MED-PC Recording file looks like

In [10]:
with open(all_med_pc_file[0]) as f:
    lines = f.readlines()
    for line in lines[:20]:
        print(line)

File: C:\MED-PC\Data\2022-10-03_11h07m_Subject 4.3v4.4.txt







Start Date: 10/03/22

End Date: 10/03/22

Subject: 4.3v4.4

Experiment: CD1_vs_C57_Comparison

Group: Cage_4

Box: 4

Start Time: 11:07:40

End Time: 11:38:55

MSN: CD1_reward_competition

A:    5399.000

D:    8000.000

E:       0.000

L:       0.000

M:       0.000

O:       0.000

T:    1860.000



- We will be extracting the first 10 or so lines that look like:

```
File: C:\MED-PC\Data\2022-05-06_12h59m_Subject 3.4 (2).txt

Start Date: 05/06/22

End Date: 05/06/22

Subject: 3.4 (2)

Experiment: Pilot of Pilot

Group: Cage 4

Box: 1

Start Time: 12:59:58

End Time: 14:02:38

MSN: levelNP_CS_reward_laserepochON1st_noshock
```

- We will just find all the lines that start with `"File", "Start Date", "End Date", "Subject", "Experiment", "Group", "Box", "Start Time", "End Time", or "MSN"`. And get the metadata from those lines. And then stop once all the metadata types have been collected

In [11]:
# This makes a nested dictionary of file paths to each individual metadata type
file_path_to_meta_data = extract.metadata.get_all_med_pc_meta_data_from_files(list_of_files=all_med_pc_file)

In [12]:
# The metadata for the first file
for key, value in file_path_to_meta_data.items():
    print("File path: {}".format(key))
    print("Metadata types and associated values: {}".format(value))
    break

File path: ./data/med_pc_recording_files/2022-10-03_11h07m_Subject 4.3v4.4.txt
Metadata types and associated values: {'File': 'C:\\MED-PC\\Data\\2022-10-03_11h07m_Subject 4.3v4.4.txt', 'Start Date': '10/03/22', 'End Date': '10/03/22', 'Subject': '4.3v4.4', 'Experiment': 'CD1_vs_C57_Comparison', 'Group': 'Cage_4', 'Box': '4', 'Start Time': '11:07:40', 'End Time': '11:38:55', 'MSN': 'CD1_reward_competition'}


## Making a Dataframe out of the Metadata

- A Dataframe is essentially a "programmable" spreadsheet. But instead of clicking on cells, you will have to tell Python how you want to interact with the spreadsheet
    - For more information: https://realpython.com/pandas-dataframe/

In [13]:
# Turning the dictionary into a Pandas Dataframe
metadata_df = pd.DataFrame.from_dict(file_path_to_meta_data, orient="index")
# Resetting the index because currently the file path is the index 
metadata_df = metadata_df.reset_index()

In [14]:
metadata_df.head()

,index,File,Start Date,End Date,Subject,Experiment,Group,Box,Start Time,End Time,MSN
0,./data/med_pc_recording_files/2022-10-03_11h07...,C:\MED-PC\Data\2022-10-03_11h07m_Subject 4.3v4...,10/03/22,10/03/22,4.3v4.4,CD1_vs_C57_Comparison,Cage_4,4,11:07:40,11:38:55,CD1_reward_competition
1,./data/med_pc_recording_files/2022-10-04_09h56...,C:\MED-PC\Data\2022-10-04_09h56m_Subject 4.1v4...,10/04/22,10/04/22,4.1v4.4,CD1_vs_C57_Comparison,Cage_4,1,09:56:16,10:29:43,CD1_reward_competition
2,./data/med_pc_recording_files/2022-10-03_09h45...,C:\MED-PC\Data\2022-10-03_09h45m_Subject 2.1v2...,10/03/22,10/03/22,2.1v2.2,CD1_vs_C57_Comparison,Cage_2,3,09:45:25,10:46:28,C57_reward_competition
3,./data/med_pc_recording_files/2022-10-03_09h45...,C:\MED-PC\Data\2022-10-03_09h45m_Subject 1.3v1...,10/03/22,10/03/22,1.3v1.4,CD1_vs_C57_Comparison,Cage_1,2,09:45:25,10:46:28,C57_reward_competition
4,./data/med_pc_recording_files/2022-10-03_11h45...,C:\MED-PC\Data\2022-10-03_11h45m_Subject 5.3v5...,10/03/22,10/03/22,5.3v5.4,CD1_vs_C57_Comparison,Cage_5,2,11:45:17,12:21:09,CD1_reward_competition


In [15]:
metadata_df.tail()

,index,File,Start Date,End Date,Subject,Experiment,Group,Box,Start Time,End Time,MSN
71,./data/med_pc_recording_files/2022-10-04_13h08...,C:\MED-PC\Data\2022-10-04_13h08m_Subject 1.2v1...,10/04/22,10/04/22,1.2v1.4,CD1_vs_C57_Comparison,Cage_1,2,13:08:08,13:43:43,C57_reward_competition
72,./data/med_pc_recording_files/2022-10-04_10h39...,C:\MED-PC\Data\2022-10-04_10h39m_Subject 2.2v2...,10/04/22,10/04/22,2.2v2.3,CD1_vs_C57_Comparison,Cage_2,2,10:39:14,11:12:10,C57_reward_competition
73,./data/med_pc_recording_files/2022-10-03_14h17...,C:\MED-PC\Data\2022-10-03_14h17m_Subject 6.2v6...,10/03/22,10/03/22,6.2v6.4,CD1_vs_C57_Comparison,Cage_6,1,14:17:01,14:52:13,CD1_reward_competition
74,./data/med_pc_recording_files/2022-10-04_13h08...,C:\MED-PC\Data\2022-10-04_13h08m_Subject 1.1v1...,10/04/22,10/04/22,1.1v1.3,CD1_vs_C57_Comparison,Cage_1,1,13:08:08,13:43:43,C57_reward_competition
75,./data/med_pc_recording_files/2022-10-04_11h42...,C:\MED-PC\Data\2022-10-04_11h42m_Subject 6.1v6...,10/04/22,10/04/22,6.1v6.3,CD1_vs_C57_Comparison,Cage_6,3,11:42:39,12:15:04,CD1_reward_competition


- Make a column that is just the file name

In [16]:
# Changing the file path to Windows format, then back to Linux format
# Then just taking the file name
metadata_df["file_name"] = metadata_df["File"].apply(lambda x: os.path.basename(pathlib.Path(pathlib.PureWindowsPath(x))).strip())

In [17]:
metadata_df

,index,File,Start Date,End Date,Subject,Experiment,Group,Box,Start Time,End Time,MSN,file_name
0,./data/med_pc_recording_files/2022-10-03_11h07...,C:\MED-PC\Data\2022-10-03_11h07m_Subject 4.3v4...,10/03/22,10/03/22,4.3v4.4,CD1_vs_C57_Comparison,Cage_4,4,11:07:40,11:38:55,CD1_reward_competition,2022-10-03_11h07m_Subject 4.3v4.4.txt
1,./data/med_pc_recording_files/2022-10-04_09h56...,C:\MED-PC\Data\2022-10-04_09h56m_Subject 4.1v4...,10/04/22,10/04/22,4.1v4.4,CD1_vs_C57_Comparison,Cage_4,1,09:56:16,10:29:43,CD1_reward_competition,2022-10-04_09h56m_Subject 4.1v4.4.txt
2,./data/med_pc_recording_files/2022-10-03_09h45...,C:\MED-PC\Data\2022-10-03_09h45m_Subject 2.1v2...,10/03/22,10/03/22,2.1v2.2,CD1_vs_C57_Comparison,Cage_2,3,09:45:25,10:46:28,C57_reward_competition,2022-10-03_09h45m_Subject 2.1v2.2.txt
3,./data/med_pc_recording_files/2022-10-03_09h45...,C:\MED-PC\Data\2022-10-03_09h45m_Subject 1.3v1...,10/03/22,10/03/22,1.3v1.4,CD1_vs_C57_Comparison,Cage_1,2,09:45:25,10:46:28,C57_reward_competition,2022-10-03_09h45m_Subject 1.3v1.4.txt
4,./data/med_pc_recording_files/2022-10-03_11h45...,C:\MED-PC\Data\2022-10-03_11h45m_Subject 5.3v5...,10/03/22,10/03/22,5.3v5.4,CD1_vs_C57_Comparison,Cage_5,2,11:45:17,12:21:09,CD1_reward_competition,2022-10-03_11h45m_Subject 5.3v5.4.txt
...,...,...,...,...,...,...,...,...,...,...,...,...
71,./data/med_pc_recording_files/2022-10-04_13h08...,C:\MED-PC\Data\2022-10-04_13h08m_Subject 1.2v1...,10/04/22,10/04/22,1.2v1.4,CD1_vs_C57_Comparison,Cage_1,2,13:08:08,13:43:43,C57_reward_competition,2022-10-04_13h08m_Subject 1.2v1.4.txt
72,./data/med_pc_recording_files/2022-10-04_10h39...,C:\MED-PC\Data\2022-10-04_10h39m_Subject 2.2v2...,10/04/22,10/04/22,2.2v2.3,CD1_vs_C57_Comparison,Cage_2,2,10:39:14,11:12:10,C57_reward_competition,2022-10-04_10h39m_Subject 2.2v2.3.txt
73,./data/med_pc_recording_files/2022-10-03_14h17...,C:\MED-PC\Data\2022-10-03_14h17m_Subject 6.2v6...,10/03/22,10/03/22,6.2v6.4,CD1_vs_C57_Comparison,Cage_6,1,14:17:01,14:52:13,CD1_reward_competition,2022-10-03_14h17m_Subject 6.2v6.4.txt
74,./data/med_pc_recording_files/2022-10-04_13h08...,C:\MED-PC\Data\2022-10-04_13h08m_Subject 1.1v1...,10/04/22,10/04/22,1.1v1.3,CD1_vs_C57_Comparison,Cage_1,1,13:08:08,13:43:43,C57_reward_competition,2022-10-04_13h08m_Subject 1.1v1.3.txt


- Getting the numbers out of the column that contains the cage information

In [18]:
metadata_df["cage"] = metadata_df["Group"].apply(lambda x: x.lower().strip("cage").strip())

In [19]:
metadata_df["cage"].head()

0    _4
1    _4
2    _2
3    _1
4    _5
Name: cage, dtype: object

- Adding a new column that combines the cage and subject ID into a new ID

In [20]:
metadata_df["cage_and_subject"] = metadata_df.apply(lambda x: "cage_{}_subject_{}".format(x["cage"], x["Subject"]), axis=1)

In [21]:
metadata_df

,index,File,Start Date,End Date,Subject,Experiment,Group,Box,Start Time,End Time,MSN,file_name,cage,cage_and_subject
0,./data/med_pc_recording_files/2022-10-03_11h07...,C:\MED-PC\Data\2022-10-03_11h07m_Subject 4.3v4...,10/03/22,10/03/22,4.3v4.4,CD1_vs_C57_Comparison,Cage_4,4,11:07:40,11:38:55,CD1_reward_competition,2022-10-03_11h07m_Subject 4.3v4.4.txt,_4,cage__4_subject_4.3v4.4
1,./data/med_pc_recording_files/2022-10-04_09h56...,C:\MED-PC\Data\2022-10-04_09h56m_Subject 4.1v4...,10/04/22,10/04/22,4.1v4.4,CD1_vs_C57_Comparison,Cage_4,1,09:56:16,10:29:43,CD1_reward_competition,2022-10-04_09h56m_Subject 4.1v4.4.txt,_4,cage__4_subject_4.1v4.4
2,./data/med_pc_recording_files/2022-10-03_09h45...,C:\MED-PC\Data\2022-10-03_09h45m_Subject 2.1v2...,10/03/22,10/03/22,2.1v2.2,CD1_vs_C57_Comparison,Cage_2,3,09:45:25,10:46:28,C57_reward_competition,2022-10-03_09h45m_Subject 2.1v2.2.txt,_2,cage__2_subject_2.1v2.2
3,./data/med_pc_recording_files/2022-10-03_09h45...,C:\MED-PC\Data\2022-10-03_09h45m_Subject 1.3v1...,10/03/22,10/03/22,1.3v1.4,CD1_vs_C57_Comparison,Cage_1,2,09:45:25,10:46:28,C57_reward_competition,2022-10-03_09h45m_Subject 1.3v1.4.txt,_1,cage__1_subject_1.3v1.4
4,./data/med_pc_recording_files/2022-10-03_11h45...,C:\MED-PC\Data\2022-10-03_11h45m_Subject 5.3v5...,10/03/22,10/03/22,5.3v5.4,CD1_vs_C57_Comparison,Cage_5,2,11:45:17,12:21:09,CD1_reward_competition,2022-10-03_11h45m_Subject 5.3v5.4.txt,_5,cage__5_subject_5.3v5.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,./data/med_pc_recording_files/2022-10-04_13h08...,C:\MED-PC\Data\2022-10-04_13h08m_Subject 1.2v1...,10/04/22,10/04/22,1.2v1.4,CD1_vs_C57_Comparison,Cage_1,2,13:08:08,13:43:43,C57_reward_competition,2022-10-04_13h08m_Subject 1.2v1.4.txt,_1,cage__1_subject_1.2v1.4
72,./data/med_pc_recording_files/2022-10-04_10h39...,C:\MED-PC\Data\2022-10-04_10h39m_Subject 2.2v2...,10/04/22,10/04/22,2.2v2.3,CD1_vs_C57_Comparison,Cage_2,2,10:39:14,11:12:10,C57_reward_competition,2022-10-04_10h39m_Subject 2.2v2.3.txt,_2,cage__2_subject_2.2v2.3
73,./data/med_pc_recording_files/2022-10-03_14h17...,C:\MED-PC\Data\2022-10-03_14h17m_Subject 6.2v6...,10/03/22,10/03/22,6.2v6.4,CD1_vs_C57_Comparison,Cage_6,1,14:17:01,14:52:13,CD1_reward_competition,2022-10-03_14h17m_Subject 6.2v6.4.txt,_6,cage__6_subject_6.2v6.4
74,./data/med_pc_recording_files/2022-10-04_13h08...,C:\MED-PC\Data\2022-10-04_13h08m_Subject 1.1v1...,10/04/22,10/04/22,1.1v1.3,CD1_vs_C57_Comparison,Cage_1,1,13:08:08,13:43:43,C57_reward_competition,2022-10-04_13h08m_Subject 1.1v1.3.txt,_1,cage__1_subject_1.1v1.3


- Making new columns so that we can use the Start date and End date in other steps

In [22]:
metadata_df["start_date_datetime"] = metadata_df["Start Date"].apply(lambda x: datetime.strptime(x, '%m/%d/%y'))
metadata_df["start_date_int"] = metadata_df["start_date_datetime"].apply(lambda x: int(x.strftime('%Y%m%d')))

metadata_df["end_date_datetime"] = metadata_df["End Date"].apply(lambda x: datetime.strptime(x, '%m/%d/%y'))
metadata_df["end_date_int"] = metadata_df["end_date_datetime"].apply(lambda x: int(x.strftime('%Y%m%d')))

In [23]:
metadata_df

,index,File,Start Date,End Date,Subject,Experiment,Group,Box,Start Time,End Time,MSN,file_name,cage,cage_and_subject,start_date_datetime,start_date_int,end_date_datetime,end_date_int
0,./data/med_pc_recording_files/2022-10-03_11h07...,C:\MED-PC\Data\2022-10-03_11h07m_Subject 4.3v4...,10/03/22,10/03/22,4.3v4.4,CD1_vs_C57_Comparison,Cage_4,4,11:07:40,11:38:55,CD1_reward_competition,2022-10-03_11h07m_Subject 4.3v4.4.txt,_4,cage__4_subject_4.3v4.4,2022-10-03,20221003,2022-10-03,20221003
1,./data/med_pc_recording_files/2022-10-04_09h56...,C:\MED-PC\Data\2022-10-04_09h56m_Subject 4.1v4...,10/04/22,10/04/22,4.1v4.4,CD1_vs_C57_Comparison,Cage_4,1,09:56:16,10:29:43,CD1_reward_competition,2022-10-04_09h56m_Subject 4.1v4.4.txt,_4,cage__4_subject_4.1v4.4,2022-10-04,20221004,2022-10-04,20221004
2,./data/med_pc_recording_files/2022-10-03_09h45...,C:\MED-PC\Data\2022-10-03_09h45m_Subject 2.1v2...,10/03/22,10/03/22,2.1v2.2,CD1_vs_C57_Comparison,Cage_2,3,09:45:25,10:46:28,C57_reward_competition,2022-10-03_09h45m_Subject 2.1v2.2.txt,_2,cage__2_subject_2.1v2.2,2022-10-03,20221003,2022-10-03,20221003
3,./data/med_pc_recording_files/2022-10-03_09h45...,C:\MED-PC\Data\2022-10-03_09h45m_Subject 1.3v1...,10/03/22,10/03/22,1.3v1.4,CD1_vs_C57_Comparison,Cage_1,2,09:45:25,10:46:28,C57_reward_competition,2022-10-03_09h45m_Subject 1.3v1.4.txt,_1,cage__1_subject_1.3v1.4,2022-10-03,20221003,2022-10-03,20221003
4,./data/med_pc_recording_files/2022-10-03_11h45...,C:\MED-PC\Data\2022-10-03_11h45m_Subject 5.3v5...,10/03/22,10/03/22,5.3v5.4,CD1_vs_C57_Comparison,Cage_5,2,11:45:17,12:21:09,CD1_reward_competition,2022-10-03_11h45m_Subject 5.3v5.4.txt,_5,cage__5_subject_5.3v5.4,2022-10-03,20221003,2022-10-03,20221003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,./data/med_pc_recording_files/2022-10-04_13h08...,C:\MED-PC\Data\2022-10-04_13h08m_Subject 1.2v1...,10/04/22,10/04/22,1.2v1.4,CD1_vs_C57_Comparison,Cage_1,2,13:08:08,13:43:43,C57_reward_competition,2022-10-04_13h08m_Subject 1.2v1.4.txt,_1,cage__1_subject_1.2v1.4,2022-10-04,20221004,2022-10-04,20221004
72,./data/med_pc_recording_files/2022-10-04_10h39...,C:\MED-PC\Data\2022-10-04_10h39m_Subject 2.2v2...,10/04/22,10/04/22,2.2v2.3,CD1_vs_C57_Comparison,Cage_2,2,10:39:14,11:12:10,C57_reward_competition,2022-10-04_10h39m_Subject 2.2v2.3.txt,_2,cage__2_subject_2.2v2.3,2022-10-04,20221004,2022-10-04,20221004
73,./data/med_pc_recording_files/2022-10-03_14h17...,C:\MED-PC\Data\2022-10-03_14h17m_Subject 6.2v6...,10/03/22,10/03/22,6.2v6.4,CD1_vs_C57_Comparison,Cage_6,1,14:17:01,14:52:13,CD1_reward_competition,2022-10-03_14h17m_Subject 6.2v6.4.txt,_6,cage__6_subject_6.2v6.4,2022-10-03,20221003,2022-10-03,20221003
74,./data/med_pc_recording_files/2022-10-04_13h08...,C:\MED-PC\Data\2022-10-04_13h08m_Subject 1.1v1...,10/04/22,10/04/22,1.1v1.3,CD1_vs_C57_Comparison,Cage_1,1,13:08:08,13:43:43,C57_reward_competition,2022-10-04_13h08m_Subject 1.1v1.3.txt,_1,cage__1_subject_1.1v1.3,2022-10-04,20221004,2022-10-04,20221004


- Making a column that is the trial number for each subject

In [24]:
subject_to_metadata_df = {}
for subject in metadata_df["cage_and_subject"].unique():
    subject_df = metadata_df[metadata_df["cage_and_subject"] == subject]
    subject_df = subject_df.sort_values("file_name", ascending=True)
    subject_df.insert(0, 'trial_number', range(1, 1 + len(subject_df)))
    subject_to_metadata_df[subject] = subject_df

In [25]:
subject_to_metadata_df = collections.OrderedDict(sorted(subject_to_metadata_df.items()))

In [26]:
all_subject_to_metadata_df = [v for k,v in subject_to_metadata_df.items()]

In [27]:
metadata_df = pd.concat(all_subject_to_metadata_df).reset_index(drop=True)

In [28]:
metadata_df.head(n=25)

,trial_number,index,File,Start Date,End Date,Subject,Experiment,Group,Box,Start Time,End Time,MSN,file_name,cage,cage_and_subject,start_date_datetime,start_date_int,end_date_datetime,end_date_int
0,1,./data/med_pc_recording_files/2022-10-03_09h45...,C:\MED-PC\Data\2022-10-03_09h45m_Subject 1.1v1...,10/03/22,10/03/22,1.1v1.2,CD1_vs_C57_Comparison,Cage_1,1,09:45:25,10:46:28,C57_reward_competition,2022-10-03_09h45m_Subject 1.1v1.2.txt,_1,cage__1_subject_1.1v1.2,2022-10-03,20221003,2022-10-03,20221003
1,2,./data/med_pc_recording_files/2022-10-04_15h37...,C:\MED-PC\Data\2022-10-04_15h37m_Subject 1.1v1...,10/04/22,10/04/22,1.1v1.2,CD1_vs_C57_Comparison,Cage_1,4,15:37:25,16:13:43,C57_reward_competition,2022-10-04_15h37m_Subject 1.1v1.2.txt,_1,cage__1_subject_1.1v1.2,2022-10-04,20221004,2022-10-04,20221004
2,1,./data/med_pc_recording_files/2022-10-03_12h32...,C:\MED-PC\Data\2022-10-03_12h32m_Subject 1.1v1...,10/03/22,10/03/22,1.1v1.3,CD1_vs_C57_Comparison,Cage_1,4,12:32:01,13:15:40,C57_reward_competition,2022-10-03_12h32m_Subject 1.1v1.3.txt,_1,cage__1_subject_1.1v1.3,2022-10-03,20221003,2022-10-03,20221003
3,2,./data/med_pc_recording_files/2022-10-04_13h08...,C:\MED-PC\Data\2022-10-04_13h08m_Subject 1.1v1...,10/04/22,10/04/22,1.1v1.3,CD1_vs_C57_Comparison,Cage_1,1,13:08:08,13:43:43,C57_reward_competition,2022-10-04_13h08m_Subject 1.1v1.3.txt,_1,cage__1_subject_1.1v1.3,2022-10-04,20221004,2022-10-04,20221004
4,1,./data/med_pc_recording_files/2022-10-03_15h04...,C:\MED-PC\Data\2022-10-03_15h04m_Subject 1.1v1...,10/03/22,10/03/22,1.1v1.4,CD1_vs_C57_Comparison,Cage_1,2,15:04:42,15:47:31,C57_reward_competition,2022-10-03_15h04m_Subject 1.1v1.4.txt,_1,cage__1_subject_1.1v1.4,2022-10-03,20221003,2022-10-03,20221003
5,2,./data/med_pc_recording_files/2022-10-04_10h39...,C:\MED-PC\Data\2022-10-04_10h39m_Subject 1.1v1...,10/04/22,10/04/22,1.1v1.4,CD1_vs_C57_Comparison,Cage_1,3,10:39:14,11:12:10,C57_reward_competition,2022-10-04_10h39m_Subject 1.1v1.4.txt,_1,cage__1_subject_1.1v1.4,2022-10-04,20221004,2022-10-04,20221004
6,1,./data/med_pc_recording_files/2022-10-03_15h04...,C:\MED-PC\Data\2022-10-03_15h04m_Subject 1.2v1...,10/03/22,10/03/22,1.2v1.3,CD1_vs_C57_Comparison,Cage_1,1,15:04:42,15:47:31,C57_reward_competition,2022-10-03_15h04m_Subject 1.2v1.3.txt,_1,cage__1_subject_1.2v1.3,2022-10-03,20221003,2022-10-03,20221003
7,2,./data/med_pc_recording_files/2022-10-04_10h39...,C:\MED-PC\Data\2022-10-04_10h39m_Subject 1.2v1...,10/04/22,10/04/22,1.2v1.3,CD1_vs_C57_Comparison,Cage_1,4,10:39:14,11:12:10,C57_reward_competition,2022-10-04_10h39m_Subject 1.2v1.3.txt,_1,cage__1_subject_1.2v1.3,2022-10-04,20221004,2022-10-04,20221004
8,1,./data/med_pc_recording_files/2022-10-03_12h32...,C:\MED-PC\Data\2022-10-03_12h32m_Subject 1.2v1...,10/03/22,10/03/22,1.2v1.4,CD1_vs_C57_Comparison,Cage_1,3,12:32:01,13:15:40,C57_reward_competition,2022-10-03_12h32m_Subject 1.2v1.4.txt,_1,cage__1_subject_1.2v1.4,2022-10-03,20221003,2022-10-03,20221003
9,2,./data/med_pc_recording_files/2022-10-04_13h08...,C:\MED-PC\Data\2022-10-04_13h08m_Subject 1.2v1...,10/04/22,10/04/22,1.2v1.4,CD1_vs_C57_Comparison,Cage_1,2,13:08:08,13:43:43,C57_reward_competition,2022-10-04_13h08m_Subject 1.2v1.4.txt,_1,cage__1_subject_1.2v1.4,2022-10-04,20221004,2022-10-04,20221004


In [29]:
metadata_df.tail(n=25)

,trial_number,index,File,Start Date,End Date,Subject,Experiment,Group,Box,Start Time,End Time,MSN,file_name,cage,cage_and_subject,start_date_datetime,start_date_int,end_date_datetime,end_date_int
51,3,./data/med_pc_recording_files/2022-10-04_14h51...,C:\MED-PC\Data\2022-10-04_14h51m_Subject 4.3v4...,10/04/22,10/04/22,4.3v4.4,CD1_vs_C57_Comparison,Cage_4,1,14:51:26,15:28:06,CD1_reward_competition,2022-10-04_14h51m_Subject 4.3v4.4.txt,_4,cage__4_subject_4.3v4.4,2022-10-04,20221004,2022-10-04,20221004
52,1,./data/med_pc_recording_files/2022-10-03_11h45...,C:\MED-PC\Data\2022-10-03_11h45m_Subject 5.1v5...,10/03/22,10/03/22,5.1v5.2,CD1_vs_C57_Comparison,Cage_5,1,11:45:17,12:21:09,CD1_reward_competition,2022-10-03_11h45m_Subject 5.1v5.2.txt,_5,cage__5_subject_5.1v5.2,2022-10-03,20221003,2022-10-03,20221003
53,2,./data/med_pc_recording_files/2022-10-04_14h12...,C:\MED-PC\Data\2022-10-04_14h12m_Subject 5.1v5...,10/04/22,10/04/22,5.1v5.2,CD1_vs_C57_Comparison,Cage_5,4,14:12:10,14:45:50,CD1_reward_competition,2022-10-04_14h12m_Subject 5.1v5.2.txt,_5,cage__5_subject_5.1v5.2,2022-10-04,20221004,2022-10-04,20221004
54,1,./data/med_pc_recording_files/2022-10-03_14h17...,C:\MED-PC\Data\2022-10-03_14h17m_Subject 5.1v5...,10/03/22,10/03/22,5.1v5.3,CD1_vs_C57_Comparison,Cage_5,4,14:17:01,14:52:13,CD1_reward_competition,2022-10-03_14h17m_Subject 5.1v5.3.txt,_5,cage__5_subject_5.1v5.3,2022-10-03,20221003,2022-10-03,20221003
55,2,./data/med_pc_recording_files/2022-10-04_11h42...,C:\MED-PC\Data\2022-10-04_11h42m_Subject 5.1v5...,10/04/22,10/04/22,5.1v5.3,CD1_vs_C57_Comparison,Cage_5,1,11:42:39,12:15:04,CD1_reward_competition,2022-10-04_11h42m_Subject 5.1v5.3.txt,_5,cage__5_subject_5.1v5.3,2022-10-04,20221004,2022-10-04,20221004
56,1,./data/med_pc_recording_files/2022-10-03_16h53...,C:\MED-PC\Data\2022-10-03_16h53m_Subject 5.1v5...,10/03/22,10/03/22,5.1v5.4,CD1_vs_C57_Comparison,Cage_5,2,16:53:00,17:37:54,CD1_reward_competition,2022-10-03_16h53m_Subject 5.1v5.4.txt,_5,cage__5_subject_5.1v5.4,2022-10-03,20221003,2022-10-03,20221003
57,2,./data/med_pc_recording_files/2022-10-04_09h07...,C:\MED-PC\Data\2022-10-04_09h07m_Subject 5.1v5...,10/04/22,10/04/22,5.1v5.4,CD1_vs_C57_Comparison,Cage_5,3,09:07:55,09:47:10,CD1_reward_competition,2022-10-04_09h07m_Subject 5.1v5.4.txt,_5,cage__5_subject_5.1v5.4,2022-10-04,20221004,2022-10-04,20221004
58,1,./data/med_pc_recording_files/2022-10-03_16h53...,C:\MED-PC\Data\2022-10-03_16h53m_Subject 5.2v5...,10/03/22,10/03/22,5.2v5.3,CD1_vs_C57_Comparison,Cage_5,1,16:53:00,17:37:54,CD1_reward_competition,2022-10-03_16h53m_Subject 5.2v5.3.txt,_5,cage__5_subject_5.2v5.3,2022-10-03,20221003,2022-10-03,20221003
59,2,./data/med_pc_recording_files/2022-10-04_09h07...,C:\MED-PC\Data\2022-10-04_09h07m_Subject 5.2v5...,10/04/22,10/04/22,5.2v5.3,CD1_vs_C57_Comparison,Cage_5,4,09:07:55,09:47:10,CD1_reward_competition,2022-10-04_09h07m_Subject 5.2v5.3.txt,_5,cage__5_subject_5.2v5.3,2022-10-04,20221004,2022-10-04,20221004
60,1,./data/med_pc_recording_files/2022-10-03_14h17...,C:\MED-PC\Data\2022-10-03_14h17m_Subject 5.2v5...,10/03/22,10/03/22,5.2v5.4,CD1_vs_C57_Comparison,Cage_5,3,14:17:01,14:52:13,CD1_reward_competition,2022-10-03_14h17m_Subject 5.2v5.4.txt,_5,cage__5_subject_5.2v5.4,2022-10-03,20221003,2022-10-03,20221003


- Getting the number of files that are associated with each subject

In [30]:
# How many files there are for each subject
metadata_df.groupby("cage_and_subject").count()

,trial_number,index,File,Start Date,End Date,Subject,Experiment,Group,Box,Start Time,End Time,MSN,file_name,cage,start_date_datetime,start_date_int,end_date_datetime,end_date_int
cage_and_subject,,,,,,,,,,,,,,,,,,
cage__1_subject_1.1v1.2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
cage__1_subject_1.1v1.3,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
cage__1_subject_1.1v1.4,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
cage__1_subject_1.2v1.3,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
cage__1_subject_1.2v1.4,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
cage__1_subject_1.3v1.4,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
cage__2_subject_2.1v2.2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
cage__2_subject_2.1v2.3,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
cage__2_subject_2.1v2.4,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2


## Looking over the MED-PC scripts

- MED-PC has scripts(lines of code that act as the instructions for how to operate the MED-PC boxes) that contain the descriptions of what each value in the recordings are. We will extract the descriptions from the MED-PC scripts so that we can label the data points in the MED-PC recordings  
- Below is a list of all the MED-PC Scripts. We will only be looking at the first one    

In [31]:
all_medpc_scripts = glob.glob("./data/**/*.MPC")

In [32]:
all_medpc_scripts

['./data/med_pc_recording_files/C57_reward_competition.MPC',
 './data/med_pc_recording_files/CD1_reward_competition.MPC']

- Example of what the MED-PC Script looks like that was ran when recording the behaviors

In [33]:
with open(all_medpc_scripts[0]) as f:
    lines = f.readlines()
    for line in lines[:100]:
        print(line)

\INPUTS



^port = 8





\OUTPUTS

^whitenoise = 1

^tone1 = 2

^tone2 = 3

^pump = 9

^fan = 16

^toneled = 17

^peled = 18







\ARRAYS



DIM P = 20000 \Port entry time stamp array



DIM Q = 250 \US delivery time stamp array (absolute)



DIM R = 250 \US time stamp array (relative to last CS)



DIM W = 2500 \ITI values used for CS



DIM S = 2500 \CS presentation values (absolute - every time yellow light turns on)



DIM N = 20000 \Port exit time stamp array



DIM K = 250 \CS type



DIM G = 250 \controlled_stimulus_seconds computer clock time (seconds on clock every time yellow light/tone turns on)



DIM H = 250 \controlled_stimulus_minutes computer clock time (minutes on clock every time yellow light/tone turns on)



DIM I = 250 \controlled_stimulus_hours computer clock time (hours every time yellow light/tone turns on)



DIM B = 2500 \port_entry_seconds computer clock time (seconds on clock every time red light turns on)



DIM F = 2500 \port_entry_minutes computer cloc

- We will be using the comments in the MED-PC script(Everything after the `\` for each line) to create a name for the variables. By default, MED-PC uses a single letter as the name of the variable(programming object that holds some information).
    - This will use the medpc2excel library found in https://github.com/cyf203/medpc2excel
- Example of the variable names and their comments in the MED-PC script that we will get the descriptive names from:
    - MED-PC will squish the first few words to create the name for every variable

```
DIM P = 20000 \Port entry time stamp array

DIM Q = 2500 \US delivery time stamp array (absolute)

DIM R = 2500 \US time stamp array (relative to last CS)

DIM W = 2500 \ITI values used for CS

DIM S = 2500 \CS presentation values (absolute - every time light turns on)

DIM N = 20000 \Port exit time stamp array

DIM K = 2500 \CS type

DIM B = 2500 \shock intensity
```

- In the MED-PC recording files, there are values that are labelled with letters. These data type of these letters is described in the MED-PC script file that we just looked at.

In [34]:
with open(all_med_pc_file[0]) as f:
    lines = f.readlines()
    for line in lines[:20]:
        print(line)

File: C:\MED-PC\Data\2022-10-03_11h07m_Subject 4.3v4.4.txt







Start Date: 10/03/22

End Date: 10/03/22

Subject: 4.3v4.4

Experiment: CD1_vs_C57_Comparison

Group: Cage_4

Box: 4

Start Time: 11:07:40

End Time: 11:38:55

MSN: CD1_reward_competition

A:    5399.000

D:    8000.000

E:       0.000

L:       0.000

M:       0.000

O:       0.000

T:    1860.000



## **NOTE: Please make sure that the corresponding `.mpc` file (aka the MED-PC script) that was ran to create the log file, is also in the same folder as the recording files. This notebook will fail to extract the data from the recording file if it is missing**

- Spreadsheet of the data from the MED-PC recording file for all the files combined into one.
- **NOTE: Each row does not correspond to the same data point. Each row represents the "n"-th data point for each category for each file. AKA, the first row is the first data point for the time the subject entered the port and the first data point for the time that the tone that is played, two seperate things. The second row is the second data point for all the categories, the third row is the third data point and so on. This repeats for all data points in a given file, then it starts over to the next file where the first row of that file is the first for all its data points.**

In [35]:
concatted_medpc_df = extract.dataframe.get_medpc_dataframe_from_list_of_files(medpc_files=all_med_pc_file)

In [36]:
concatted_medpc_df.head()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(G)controlled_stimulus_secondscomputer,(H)controlled_stimulus_minutescomputer,(I)controlled_stimulus_hourscomputer,(B)port_entry_secondscomputer,(F)port_entry_minutescomputer,(J)port_entry_hourscomputer,date,subject,file_path
0,0.02,64.0,399.0,0.0,60.01,0.43,1.0,55.0,8.0,11.0,55.0,7.0,11.0,20221003,4.3v4.4,./data/med_pc_recording_files/2022-10-03_11h07...
1,12.13,144.0,399.0,0.0,140.01,12.20,1.0,15.0,10.0,11.0,7.0,8.0,11.0,20221003,4.3v4.4,./data/med_pc_recording_files/2022-10-03_11h07...
2,12.22,234.0,399.0,0.0,230.01,12.38,1.0,45.0,11.0,11.0,7.0,8.0,11.0,20221003,4.3v4.4,./data/med_pc_recording_files/2022-10-03_11h07...
3,12.49,314.0,399.0,0.0,310.01,12.99,1.0,5.0,13.0,11.0,8.0,8.0,11.0,20221003,4.3v4.4,./data/med_pc_recording_files/2022-10-03_11h07...
4,19.87,389.0,399.0,0.0,385.01,19.91,1.0,20.0,14.0,11.0,15.0,8.0,11.0,20221003,4.3v4.4,./data/med_pc_recording_files/2022-10-03_11h07...


In [37]:
concatted_medpc_df.tail()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(G)controlled_stimulus_secondscomputer,(H)controlled_stimulus_minutescomputer,(I)controlled_stimulus_hourscomputer,(B)port_entry_secondscomputer,(F)port_entry_minutescomputer,(J)port_entry_hourscomputer,date,subject,file_path
6003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,NaN,NaN,20221004,6.1v6.3,./data/med_pc_recording_files/2022-10-04_11h42...
6004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,20221004,6.1v6.3,./data/med_pc_recording_files/2022-10-04_11h42...
6005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3024.0,NaN,NaN,20221004,6.1v6.3,./data/med_pc_recording_files/2022-10-04_11h42...
6006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0,NaN,NaN,20221004,6.1v6.3,./data/med_pc_recording_files/2022-10-04_11h42...
6007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,NaN,NaN,20221004,6.1v6.3,./data/med_pc_recording_files/2022-10-04_11h42...


- Combining the recording and the metadata into one dataframe

In [38]:
recording_and_metadata_df = metadata_df.merge(concatted_medpc_df, right_on='file_path', left_on='index')

In [39]:
recording_and_metadata_df

,trial_number,index,File,Start Date,End Date,Subject,Experiment,Group,Box,Start Time,...,(K)CStype,(G)controlled_stimulus_secondscomputer,(H)controlled_stimulus_minutescomputer,(I)controlled_stimulus_hourscomputer,(B)port_entry_secondscomputer,(F)port_entry_minutescomputer,(J)port_entry_hourscomputer,date,subject,file_path
0,1,./data/med_pc_recording_files/2022-10-03_09h45...,C:\MED-PC\Data\2022-10-03_09h45m_Subject 1.1v1...,10/03/22,10/03/22,1.1v1.2,CD1_vs_C57_Comparison,Cage_1,1,09:45:25,...,1.0,28.0,16.0,10.0,29.0,15.0,10.0,20221003,1.1v1.2,./data/med_pc_recording_files/2022-10-03_09h45...
1,1,./data/med_pc_recording_files/2022-10-03_09h45...,C:\MED-PC\Data\2022-10-03_09h45m_Subject 1.1v1...,10/03/22,10/03/22,1.1v1.2,CD1_vs_C57_Comparison,Cage_1,1,09:45:25,...,1.0,49.0,17.0,10.0,39.0,15.0,10.0,20221003,1.1v1.2,./data/med_pc_recording_files/2022-10-03_09h45...
2,1,./data/med_pc_recording_files/2022-10-03_09h45...,C:\MED-PC\Data\2022-10-03_09h45m_Subject 1.1v1...,10/03/22,10/03/22,1.1v1.2,CD1_vs_C57_Comparison,Cage_1,1,09:45:25,...,1.0,18.0,19.0,10.0,39.0,15.0,10.0,20221003,1.1v1.2,./data/med_pc_recording_files/2022-10-03_09h45...
3,1,./data/med_pc_recording_files/2022-10-03_09h45...,C:\MED-PC\Data\2022-10-03_09h45m_Subject 1.1v1...,10/03/22,10/03/22,1.1v1.2,CD1_vs_C57_Comparison,Cage_1,1,09:45:25,...,1.0,38.0,20.0,10.0,39.0,15.0,10.0,20221003,1.1v1.2,./data/med_pc_recording_files/2022-10-03_09h45...
4,1,./data/med_pc_recording_files/2022-10-03_09h45...,C:\MED-PC\Data\2022-10-03_09h45m_Subject 1.1v1...,10/03/22,10/03/22,1.1v1.2,CD1_vs_C57_Comparison,Cage_1,1,09:45:25,...,1.0,53.0,21.0,10.0,40.0,15.0,10.0,20221003,1.1v1.2,./data/med_pc_recording_files/2022-10-03_09h45...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496832,2,./data/med_pc_recording_files/2022-10-04_14h12...,C:\MED-PC\Data\2022-10-04_14h12m_Subject 6.3v6...,10/04/22,10/04/22,6.3v6.4,CD1_vs_C57_Comparison,Cage_6,1,14:12:10,...,NaN,NaN,NaN,NaN,20.0,NaN,NaN,20221004,6.3v6.4,./data/med_pc_recording_files/2022-10-04_14h12...
496833,2,./data/med_pc_recording_files/2022-10-04_14h12...,C:\MED-PC\Data\2022-10-04_14h12m_Subject 6.3v6...,10/04/22,10/04/22,6.3v6.4,CD1_vs_C57_Comparison,Cage_6,1,14:12:10,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,20221004,6.3v6.4,./data/med_pc_recording_files/2022-10-04_14h12...
496834,2,./data/med_pc_recording_files/2022-10-04_14h12...,C:\MED-PC\Data\2022-10-04_14h12m_Subject 6.3v6...,10/04/22,10/04/22,6.3v6.4,CD1_vs_C57_Comparison,Cage_6,1,14:12:10,...,NaN,NaN,NaN,NaN,2059.0,NaN,NaN,20221004,6.3v6.4,./data/med_pc_recording_files/2022-10-04_14h12...
496835,2,./data/med_pc_recording_files/2022-10-04_14h12...,C:\MED-PC\Data\2022-10-04_14h12m_Subject 6.3v6...,10/04/22,10/04/22,6.3v6.4,CD1_vs_C57_Comparison,Cage_6,1,14:12:10,...,NaN,NaN,NaN,NaN,19.0,NaN,NaN,20221004,6.3v6.4,./data/med_pc_recording_files/2022-10-04_14h12...


In [40]:
recording_and_metadata_df

,trial_number,index,File,Start Date,End Date,Subject,Experiment,Group,Box,Start Time,...,(K)CStype,(G)controlled_stimulus_secondscomputer,(H)controlled_stimulus_minutescomputer,(I)controlled_stimulus_hourscomputer,(B)port_entry_secondscomputer,(F)port_entry_minutescomputer,(J)port_entry_hourscomputer,date,subject,file_path
0,1,./data/med_pc_recording_files/2022-10-03_09h45...,C:\MED-PC\Data\2022-10-03_09h45m_Subject 1.1v1...,10/03/22,10/03/22,1.1v1.2,CD1_vs_C57_Comparison,Cage_1,1,09:45:25,...,1.0,28.0,16.0,10.0,29.0,15.0,10.0,20221003,1.1v1.2,./data/med_pc_recording_files/2022-10-03_09h45...
1,1,./data/med_pc_recording_files/2022-10-03_09h45...,C:\MED-PC\Data\2022-10-03_09h45m_Subject 1.1v1...,10/03/22,10/03/22,1.1v1.2,CD1_vs_C57_Comparison,Cage_1,1,09:45:25,...,1.0,49.0,17.0,10.0,39.0,15.0,10.0,20221003,1.1v1.2,./data/med_pc_recording_files/2022-10-03_09h45...
2,1,./data/med_pc_recording_files/2022-10-03_09h45...,C:\MED-PC\Data\2022-10-03_09h45m_Subject 1.1v1...,10/03/22,10/03/22,1.1v1.2,CD1_vs_C57_Comparison,Cage_1,1,09:45:25,...,1.0,18.0,19.0,10.0,39.0,15.0,10.0,20221003,1.1v1.2,./data/med_pc_recording_files/2022-10-03_09h45...
3,1,./data/med_pc_recording_files/2022-10-03_09h45...,C:\MED-PC\Data\2022-10-03_09h45m_Subject 1.1v1...,10/03/22,10/03/22,1.1v1.2,CD1_vs_C57_Comparison,Cage_1,1,09:45:25,...,1.0,38.0,20.0,10.0,39.0,15.0,10.0,20221003,1.1v1.2,./data/med_pc_recording_files/2022-10-03_09h45...
4,1,./data/med_pc_recording_files/2022-10-03_09h45...,C:\MED-PC\Data\2022-10-03_09h45m_Subject 1.1v1...,10/03/22,10/03/22,1.1v1.2,CD1_vs_C57_Comparison,Cage_1,1,09:45:25,...,1.0,53.0,21.0,10.0,40.0,15.0,10.0,20221003,1.1v1.2,./data/med_pc_recording_files/2022-10-03_09h45...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496832,2,./data/med_pc_recording_files/2022-10-04_14h12...,C:\MED-PC\Data\2022-10-04_14h12m_Subject 6.3v6...,10/04/22,10/04/22,6.3v6.4,CD1_vs_C57_Comparison,Cage_6,1,14:12:10,...,NaN,NaN,NaN,NaN,20.0,NaN,NaN,20221004,6.3v6.4,./data/med_pc_recording_files/2022-10-04_14h12...
496833,2,./data/med_pc_recording_files/2022-10-04_14h12...,C:\MED-PC\Data\2022-10-04_14h12m_Subject 6.3v6...,10/04/22,10/04/22,6.3v6.4,CD1_vs_C57_Comparison,Cage_6,1,14:12:10,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,20221004,6.3v6.4,./data/med_pc_recording_files/2022-10-04_14h12...
496834,2,./data/med_pc_recording_files/2022-10-04_14h12...,C:\MED-PC\Data\2022-10-04_14h12m_Subject 6.3v6...,10/04/22,10/04/22,6.3v6.4,CD1_vs_C57_Comparison,Cage_6,1,14:12:10,...,NaN,NaN,NaN,NaN,2059.0,NaN,NaN,20221004,6.3v6.4,./data/med_pc_recording_files/2022-10-04_14h12...
496835,2,./data/med_pc_recording_files/2022-10-04_14h12...,C:\MED-PC\Data\2022-10-04_14h12m_Subject 6.3v6...,10/04/22,10/04/22,6.3v6.4,CD1_vs_C57_Comparison,Cage_6,1,14:12:10,...,NaN,NaN,NaN,NaN,19.0,NaN,NaN,20221004,6.3v6.4,./data/med_pc_recording_files/2022-10-04_14h12...


## Getting the cage numbers and the dates so that we can include it in the file name

- We will get the metadata from the recording files and use it to name the files we will create. This will help organize the files and make it easy to know where each file came from 

- Getting the group numbers(original cage names)

In [41]:
# removing blank spaces
group_numbers = ["_".join(number.split()) for number in recording_and_metadata_df["Group"].unique() if number]
# sorting numbers
group_numbers = sorted(group_numbers)
group_numbers_for_title = "_".join(group_numbers)

In [42]:
group_numbers_for_title

'Cage_1_Cage_2_Cage_3_Cage_4_Cage_5_Cage_6'

- Getting the cage numbers

In [43]:
# removing blank spaces
cage_numbers = ["_".join(number.split()) for number in recording_and_metadata_df["cage"].unique() if number]
# sorting numbers
cage_numbers = sorted(cage_numbers)
cage_numbers_for_title = "_".join(cage_numbers)

In [44]:
cage_numbers_for_title

'_1__2__3__4__5__6'

- Getting the dates

In [45]:
# Getting the first and last recording date to get a range
earliest_date = recording_and_metadata_df["end_date_int"].min()
latest_date = recording_and_metadata_df["end_date_int"].max()

In [46]:
earliest_date

20221003

In [47]:
latest_date

20221004

- Getting the subject names

In [48]:
recording_and_metadata_df["Subject"].unique()

array(['1.1v1.2', '1.1v1.3', '1.1v1.4', '1.2v1.3', '1.2v1.4', '1.3v1.4',
       '2.1v2.2', '2.1v2.3', '2.1v2.4', '2.2v2.3', '2.2v2.4', '2.3v2.4',
       '3.1v3.2', '3.1v3.3', '3.1v3.4', '3.2v3.3', '3.2v3.4', '3.3v3.4',
       '4.1v4.2', '4.1v4.3', '4.1v4.4', '4.2v4.3', '4.2v4.4', '4.3v4.4',
       '5.1v5.2', '5.1v5.3', '5.1v5.4', '5.2v5.3', '5.2v5.4', '5.3v5.4',
       '6.1v6.2', '6.1v6.3', '6.1v6.4', '6.2v6.3', '6.2v6.4', '6.3v6.4'],
      dtype=object)

In [49]:
# removing blank spaces
subject_numbers = ["subject-" + number for number in recording_and_metadata_df["Subject"].unique() if number]
# sorting numbers
subject_numbers = sorted(subject_numbers)
subject_numbers_for_title = "_".join(subject_numbers)

In [50]:
subject_numbers_for_title

'subject-1.1v1.2_subject-1.1v1.3_subject-1.1v1.4_subject-1.2v1.3_subject-1.2v1.4_subject-1.3v1.4_subject-2.1v2.2_subject-2.1v2.3_subject-2.1v2.4_subject-2.2v2.3_subject-2.2v2.4_subject-2.3v2.4_subject-3.1v3.2_subject-3.1v3.3_subject-3.1v3.4_subject-3.2v3.3_subject-3.2v3.4_subject-3.3v3.4_subject-4.1v4.2_subject-4.1v4.3_subject-4.1v4.4_subject-4.2v4.3_subject-4.2v4.4_subject-4.3v4.4_subject-5.1v5.2_subject-5.1v5.3_subject-5.1v5.4_subject-5.2v5.3_subject-5.2v5.4_subject-5.3v5.4_subject-6.1v6.2_subject-6.1v6.3_subject-6.1v6.4_subject-6.2v6.3_subject-6.2v6.4_subject-6.3v6.4'

- Getting the experiment name

In [51]:
experiment_names = ["_".join(name.upper().split()) for name in recording_and_metadata_df["Experiment"].unique() if name]
# sorting experiment_names
experiment_names = list(set(sorted(experiment_names)))
experiment_names_for_title = "AND".join(experiment_names)

In [52]:
experiment_names_for_title

'CD1_VS_C57_COMPARISON'

- Getting the box numbers

In [53]:
# removing blank spaces
box_numbers = ["_".join(number.split()) for number in recording_and_metadata_df["Box"].unique() if number]
# sorting numbers
box_numbers = sorted(box_numbers)
box_numbers_for_title = "_".join(box_numbers)

In [54]:
box_numbers_for_title

'1_2_3_4'

- Getting the script names

In [55]:
# removing blank spaces
script_names = ["_".join(name.split()) for name in recording_and_metadata_df["MSN"].unique() if name]
# sorting names
script_names = sorted(script_names)
script_names_for_title = "_".join(script_names)

In [56]:
script_names_for_title

'C57_reward_competition_CD1_reward_competition'

## Saving the dataframes(spreadsheets to files) with the metadata as part of the name

- Making necessary directories
    - If you want to use any of the other metadata as part of the name, you will have to swap out the variables in the `format()` and change the name of the folder to match your new name. The variable names are the word that's in front of the `=` at the last line of each cell. The `{}` are where the metadata variables will be inserted into the file name. For more information on formatting strings: https://www.w3schools.com/python/ref_string_format.asp
    - You can also just manually rename the files by replacing everything in `""` and removing the `.format()` part
- **NOTE: You may get an error that the file does not exist. If this is the case, it could be the file name is too long(an issue that may happen when using Jupyter Notebooks on Windows)**

In [57]:
output_directory = "./proc/extracted_recording_data_and_metadata/experiment_{}_cage_{}_date_{}_{}".format(experiment_names_for_title, cage_numbers_for_title, earliest_date, latest_date)

In [58]:
output_directory

'./proc/extracted_recording_data_and_metadata/experiment_CD1_VS_C57_COMPARISON_cage__1__2__3__4__5__6_date_20221003_20221004'

In [59]:
os.makedirs(output_directory, exist_ok=True)

In [60]:
metadata_df.to_csv(os.path.join(output_directory, "metadata_cage_{}_date_{}_{}.csv".format(cage_numbers_for_title, earliest_date, latest_date)))
# metadata_df.to_excel(os.path.join(output_directory, "metadata_cage_{}_date_{}_{}.xlsx".format(cage_numbers_for_title, earliest_date, latest_date)))

In [61]:
concatted_medpc_df.to_csv(os.path.join(output_directory, "MEDPC_recording_cage_{}_date_{}_{}.csv".format(cage_numbers_for_title, earliest_date, latest_date)))
# concatted_medpc_df.to_excel(os.path.join(output_directory, "MEDPC_recording_cage_{}_date_{}_{}.xlsx".format(cage_numbers_for_title, earliest_date, latest_date)))

In [62]:
recording_and_metadata_df.to_csv(os.path.join(output_directory, "recording_metadata_cage_{}_date_{}_{}.csv".format(cage_numbers_for_title, earliest_date, latest_date)))
# recording_and_metadata_df.to_excel(os.path.join(output_directory, "recording_and_metadata_cage_{}_date_{}_{}.xlsx".format(cage_numbers_for_title, earliest_date, latest_date)))